In [ ]:
import os
import cv2
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import json
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.models import Model
from keras.layers import Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Activation,GlobalAveragePooling2D, Dense, BatchNormalization, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import SGD

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Set the path to your dataset
dataset_path = "/content/drive/MyDrive/cotton_data/cotton"

# Load the dataset
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_path,
    image_size=(128, 128),  # Resize images to a consistent size
    batch_size=32,          # Number of images per batch
    label_mode='categorical',  # or 'int' for numeric labels
    shuffle=True            # Shuffle the dataset
)

# Check the dataset
for images, labels in dataset.take(1):
    print(images.shape)  # (batch_size, height, width, channels)
    print(labels.shape)  # (batch_size, number_of_classes)

Found 1727 files belonging to 4 classes.
(32, 128, 128, 3)
(32, 4)


In [ ]:
import tensorflow as tf

# Set the path to your dataset
dataset_path = "/content/drive/MyDrive/cotton_data/cotton"

# Load the training dataset (80%)
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_path,
    image_size=(128, 128),        # Resize images
    batch_size=32,                # Number of images per batch
    validation_split=0.2,         # Reserve 20% for validation (or testing)
    subset='training',            # This is the training set
    seed=123,                     # Reproducibility
    shuffle=True                  # Shuffle the dataset
)

# Load the testing dataset (20%)
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_path,
    image_size=(128, 128),        # Resize images
    batch_size=32,                # Number of images per batch
    validation_split=0.2,         # Reserve 20% for validation (or testing)
    subset='validation',          # This is the validation/test set
    seed=123,                     # Reproducibility
    shuffle=True                  # Shuffle the dataset
)

# Check the datasets
for images, labels in train_dataset.take(1):
    print(f"Train: {images.shape}, {labels.shape}")

for images, labels in test_dataset.take(1):
    print(f"Test: {images.shape}, {labels.shape}")

Found 1727 files belonging to 4 classes.
Using 1382 files for training.
Found 1727 files belonging to 4 classes.
Using 345 files for validation.
Train: (32, 128, 128, 3), (32,)
Test: (32, 128, 128, 3), (32,)


In [ ]:
# Import the required modules
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.applications.resnet50 import ResNet50

# Create the base model from the pre-trained model ResNet50
base_model = ResNet50(weights='imagenet', include_top=False)

# Add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)

# Add a fully-connected layer
x = Dense(1024, activation='relu')(x)

# Add a logistic layer with the number of classes in your dataset
predictions = Dense(4, activation='softmax')(x)  # Adjust the number of classes as needed

# This is the model we will train
resnet50_model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the ResNet50 layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model (should be done *after* setting layers to non-trainable)
resnet50_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


In [ ]:
#Getting model's summary
resnet50_model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, None, None, 3)  │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_pad (ZeroPadding2D) │ (None, None, None, 3)  │              0 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_conv (Conv2D)       │ (None, None, None, 64) │          9,472 │ conv1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_bn                  │ (None, None, None, 64) │            256 │ conv1_conv[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_relu (Activation)   │ (None, None, None, 64) │              0 │ conv1_bn[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pad (ZeroPadding2D) │ (None, None, None, 64) │              0 │ conv1_relu[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pool (MaxPooling2D) │ (None, None, None, 64) │              0 │ pool1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_conv       │ (None, None, None, 64) │          4,160 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_bn         │ (None, None, None, 64) │            256 │ conv2_block1_1_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_relu       │ (None, None, None, 64) │              0 │ conv2_block1_1_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_conv       │ (None, None, None, 64) │         36,928 │ conv2_block1_1_relu[0… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_bn         │ (None, None, None, 64) │            256 │ conv2_block1_2_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_relu       │ (None, None, None, 64) │              0 │ conv2_block1_2_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_0_conv       │ (None, None, None,     │         16,640 │ pool1_pool[0][0]       │
│ (Conv2D)                  │ 256)                   │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_3_conv       │ (None, None, None,     │         16,640 │ conv2_block1_2_relu[0… │
│ (Conv2D)                  │ 256)                   │                │                        │
├──────────────────────

 Total params: 25,689,988 (98.00 MB)

 Trainable params: 2,102,276 (8.02 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [ ]:
#Specifing epochs & batch size for vggnet16
epochs = 40
batch_size = 32
image_height = 128
image_width = 128

In [ ]:
#Creating an object of ImageDataGenerator for augmenting training dataset
train_datagen = ImageDataGenerator(rescale= 1./255,
rotation_range=10,
width_shift_range=0.1,
height_shift_range=0.1,
shear_range=0.1,
zoom_range=0.1,
horizontal_flip=True,
fill_mode='nearest')

#Creating an object of ImageDataGenerator for augmenting test dataset
test_datagen = ImageDataGenerator(rescale= 1./255)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define image size and batch size
image_size = (128, 128)   # VGG19 requires 128*128 input
batch_size = 32           # Number of images to process in a batch

# Define the train data generator with augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,           # Rescale pixel values to [0, 1]
    shear_range=0.2,          # Apply random shear transformations
    zoom_range=0.2,           # Apply random zoom
    horizontal_flip=True,      # Randomly flip images horizontally
    validation_split=0.2      # Split 20% of the data for validation
)

# Define the test/validation data generator (only rescale)
test_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# Set dataset directory path
dataset_path = "/content/drive/MyDrive/cotton_data/cotton"

# Create train data generator
train_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=image_size,      # Resize all images to 128*128
    batch_size=batch_size,
    class_mode='categorical',    # For multi-class classification
    subset='training',           # Use this for the training data
    shuffle=True                 # Shuffle the data for better training
)

# Create validation data generator
validation_generator = test_datagen.flow_from_directory(
    dataset_path,
    target_size=image_size,      # Resize all images to 128*128
    batch_size=batch_size,
    class_mode='categorical',    # For multi-class classification
    subset='validation',         # Use this for the validation data
    shuffle=False                # No need to shuffle the validation data
)

# Check the generators
print(f"Train Generator: {len(train_generator)} batches")
print(f"Validation Generator: {len(validation_generator)} batches")


Found 1382 images belonging to 4 classes.
Found 344 images belonging to 4 classes.
Train Generator: 44 batches
Validation Generator: 11 batches


In [ ]:
# Define image size and batch size
image_size = (128, 128)
batch_size = 32           # Number of images to process in a batch

# Define the test data generator (only rescale, no augmentation)
test_datagen = ImageDataGenerator(rescale=1./255)  # Rescale pixel values to [0, 1]

# Set dataset directory path for test data
test_dataset_path = "/content/drive/MyDrive/cotton_data/cotton"

# Create test data generator
test_generator = test_datagen.flow_from_directory(
    test_dataset_path,
    target_size=image_size,      # Resize all images to 128*128
    batch_size=batch_size,
    class_mode='categorical',    # For multi-class classification
    shuffle=False                # Do not shuffle the test data
)

# Check the test generator
print(f"Test Generator: {len(test_generator)} batches")

Found 1726 images belonging to 4 classes.
Test Generator: 54 batches


In [ ]:
resnet50_history = resnet50_model.fit(train_generator, epochs=epochs, validation_data=validation_generator)

Epoch 1/40


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


44/44 ━━━━━━━━━━━━━━━━━━━━ 519s 10s/step - accuracy: 0.2974 - loss: 1.4256 - val_accuracy: 0.3692 - val_loss: 1.3010
Epoch 2/40
44/44 ━━━━━━━━━━━━━━━━━━━━ 170s 3s/step - accuracy: 0.4359 - loss: 1.2292 - val_accuracy: 0.4884 - val_loss: 1.1803
Epoch 3/40
44/44 ━━━━━━━━━━━━━━━━━━━━ 164s 3s/step - accuracy: 0.4808 - loss: 1.1870 - val_accuracy: 0.5291 - val_loss: 1.1714
Epoch 4/40
44/44 ━━━━━━━━━━━━━━━━━━━━ 127s 3s/step - accuracy: 0.4963 - loss: 1.1525 - val_accuracy: 0.4593 - val_loss: 1.1954
Epoch 5/40
44/44 ━━━━━━━━━━━━━━━━━━━━ 143s 3s/step - accuracy: 0.4872 - loss: 1.1530 - val_accuracy: 0.5145 - val_loss: 1.1294
Epoch 6/40
44/44 ━━━━━━━━━━━━━━━━━━━━ 182s 3s/step - accuracy: 0.5191 - loss: 1.1073 - val_accuracy: 0.4738 - val_loss: 1.2647
Epoch 7/40
44/44 ━━━━━━━━━━━━━━━━━━━━ 141s 3s/step - accuracy: 0.5495 - loss: 1.0854 - val_accuracy: 0.5698 - val_loss: 1.1139
Epoch 8/40
44/44 ━━━━━━━━━━━━━━━━━━━━ 128s 3s/step - accuracy: 0.5272 - loss: 1.0739 - val_accuracy: 0.4855 - val_loss: 1

KeyboardInterrupt: 

In [ ]:
from sklearn.metrics import classification_report

# predictions from the test data
y_pred = resnet50_model.predict(test_generator)

# transform class labels of the predictions
y_pred_classes = np.argmax(y_pred, axis=1)

# Real classes
y_true = test_generator.classes

# Create classification report
report = classification_report(y_true, y_pred_classes, target_names=test_generator.class_indices.keys())

# print report
print(report)

/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


54/54 ━━━━━━━━━━━━━━━━━━━━ 124s 2s/step
                  precision    recall  f1-score   support

bacterial_blight       0.56      0.19      0.28       448
      curl_virus       0.46      0.76      0.57       427
  fussarium_wilt       0.65      0.81      0.72       426
         healthy       0.73      0.59      0.65       425

        accuracy                           0.58      1726
       macro avg       0.60      0.59      0.56      1726
    weighted avg       0.60      0.58      0.55      1726

